En la lección de hoy aprendimos como crearnos una clase que nos permita limpiar los datos obtenidos de la API.
En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [1]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

import requests
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
#pd.options.display.float_format = '{:.2f}'.format

In [2]:
class Energias:

    def __init__(self, inicio, fin):
        self.inicio =  inicio
        self.fin = fin

    def extraer_tipos(self):
        datos_energia = pd.DataFrame()
        for i in range(self.inicio, self.fin):
            url = f"https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={i}-01-01T00:00&end_date={i}-12-31T23:59&time_trunc=year"
            response = requests.get(url=url)
            datos = response.json()

            for e in range(len(datos["included"])):
                x = pd.DataFrame(datos["included"][e]["attributes"]["values"])
                x["tipo_energia"] = datos["included"][e]["attributes"]["title"]

                datos_energia = pd.concat([datos_energia, x], axis=0, ignore_index = True)

        return datos_energia
    
    def energia_comunidades(self, claves):
        self.claves = claves
        datos_energia = pd.DataFrame()
        for i in range(self.inicio, self.fin):
            for k,v in claves.items():
                url = f"https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={i}-01-01T00:00&end_date={i}-12-31T23:59&time_trunc=year&geo_trunc=electric_system&geo_limit=ccaa&geo_ids={v}"
                response = requests.get(url=url)
                datos = response.json()

                for e in range(len(datos["included"])):
                    x = pd.DataFrame(datos["included"][e]["attributes"]["values"])
                    x["tipo_energia"] = datos["included"][e]["attributes"]["title"]
                    x["comunidad"] = k
                    x["cod_comunidad"] = v

                    datos_energia = pd.concat([datos_energia, x], axis=0, ignore_index = True)

        return datos_energia
    
    def redondear_valores(self, df):
        self.df =df
        df["value"] = round(df["value"], 2)
        df["percentage"] = round(df["percentage"], 2)
        return df 
    
    def sacar_fecha(self, df):
        self.df =df
        df["fecha"] = df["datetime"]
        df["fecha"] = df["fecha"].apply(lambda tiempo: tiempo.split("T")[0])
        #df["fecha"] = df["fecha"].apply(pd.to_datetime)
        df["fecha"] = df.apply(lambda datafecha: datetime.strptime(datafecha["fecha"], "%Y-%m-%d"), axis = 1)
        return df 
    
    def guardar_datos(self, df, ruta, nombre):
        self.ruta = ruta
        self.df =df
        self.nombre = nombre

        df.to_pickle(ruta + nombre + ".pkl")
        df.to_csv(ruta + nombre + ".csv")

In [3]:
cod_comunidades = {'Ceuta': 8744,
                    'Melilla': 8745,
                    'Andalucía': 4,
                    'Aragón': 5,
                    'Cantabria': 6,
                    'Castilla - La Mancha': 7,
                    'Castilla y León': 8,
                    'Cataluña': 9,
                    'País Vasco': 10,
                    'Principado de Asturias': 11,
                    'Comunidad de Madrid': 13,
                    'Comunidad Foral de Navarra': 14,
                    'Comunitat Valenciana': 15,
                    'Extremadura': 16,
                    'Galicia': 17,
                    'Illes Balears': 8743,
                    'Canarias': 8742,
                    'Región de Murcia': 21,
                    'La Rioja': 20}

In [4]:
energy = Energias(2011, 2023)

In [5]:
energias_renovables = energy.extraer_tipos()

In [6]:
energias_renovables.head()

,value,percentage,datetime,tipo_energia
0,8.666705e+07,0.310241,2011-01-01T00:00:00.000+01:00,Renovable
1,1.926868e+08,0.689759,2011-01-01T00:00:00.000+01:00,No renovable
2,8.533872e+07,0.301424,2012-01-01T00:00:00.000+01:00,Renovable
3,1.977799e+08,0.698576,2012-01-01T00:00:00.000+01:00,No renovable
4,1.097577e+08,0.401622,2013-01-01T00:00:00.000+01:00,Renovable


In [12]:
tipos_por_comunidades = energy.energia_comunidades(cod_comunidades)

In [13]:
tipos_por_comunidades.head()

,value,percentage,datetime,tipo_energia,comunidad,cod_comunidad
0,2.029729e+05,1.000000,2011-01-01T00:00:00.000+01:00,No renovable,Ceuta,8744
1,3.589082e+03,0.016703,2011-01-01T00:00:00.000+01:00,Renovable,Melilla,8745
2,2.112900e+05,0.983297,2011-01-01T00:00:00.000+01:00,No renovable,Melilla,8745
3,1.099692e+07,0.284857,2011-01-01T00:00:00.000+01:00,Renovable,Andalucía,4
4,2.760811e+07,0.715143,2011-01-01T00:00:00.000+01:00,No renovable,Andalucía,4


In [7]:
energias_renovables_round = energy.redondear_valores(energias_renovables)

In [8]:
energias_renovables_round.head(1)

,value,percentage,datetime,tipo_energia
0,86667052.85,0.31,2011-01-01T00:00:00.000+01:00,Renovable


In [14]:
tipos_por_comunidades_round = energy.redondear_valores(tipos_por_comunidades)

In [15]:
tipos_por_comunidades_round.head()

,value,percentage,datetime,tipo_energia,comunidad,cod_comunidad
0,202972.89,1.00,2011-01-01T00:00:00.000+01:00,No renovable,Ceuta,8744
1,3589.08,0.02,2011-01-01T00:00:00.000+01:00,Renovable,Melilla,8745
2,211290.04,0.98,2011-01-01T00:00:00.000+01:00,No renovable,Melilla,8745
3,10996921.24,0.28,2011-01-01T00:00:00.000+01:00,Renovable,Andalucía,4
4,27608105.11,0.72,2011-01-01T00:00:00.000+01:00,No renovable,Andalucía,4


In [9]:
energias_renovables_round.head()

,value,percentage,datetime,tipo_energia
0,8.666705e+07,0.31,2011-01-01T00:00:00.000+01:00,Renovable
1,1.926868e+08,0.69,2011-01-01T00:00:00.000+01:00,No renovable
2,8.533872e+07,0.30,2012-01-01T00:00:00.000+01:00,Renovable
3,1.977799e+08,0.70,2012-01-01T00:00:00.000+01:00,No renovable
4,1.097577e+08,0.40,2013-01-01T00:00:00.000+01:00,Renovable


In [10]:
energias_renovables_fecha = energy.sacar_fecha(energias_renovables_round)

In [11]:
energias_renovables_fecha.head()

,value,percentage,datetime,tipo_energia,fecha
0,8.666705e+07,0.31,2011-01-01T00:00:00.000+01:00,Renovable,2011-01-01
1,1.926868e+08,0.69,2011-01-01T00:00:00.000+01:00,No renovable,2011-01-01
2,8.533872e+07,0.30,2012-01-01T00:00:00.000+01:00,Renovable,2012-01-01
3,1.977799e+08,0.70,2012-01-01T00:00:00.000+01:00,No renovable,2012-01-01
4,1.097577e+08,0.40,2013-01-01T00:00:00.000+01:00,Renovable,2013-01-01


In [17]:
tipos_por_comunidades_fecha = energy.sacar_fecha(tipos_por_comunidades_round)

In [18]:
guardado_tipos = energy.guardar_datos(energias_renovables_fecha, "datos/", "tipo_energia")

In [19]:
guardado_comunidades = energy.guardar_datos(tipos_por_comunidades_fecha, "datos/", "datos_comunidades")